In [2]:
import pandas as pd
import os
import numpy as np
from lexicalrichness import LexicalRichness
import textstat
import re
import nltk
from collections import Counter
from scipy.stats import ttest_ind
from scipy.stats import levene
from scipy.stats import wald
from scipy.stats import mannwhitneyu
import itertools

In [3]:
inpath2="C:/Users/pcrrt/Applied NLP Final Project/Data/Intermediate Data/"

In [4]:
df=pd.read_csv(inpath2+'final v4.csv')

In [5]:
df['text_len']=df['text'].apply(lambda x: len(x))

In [6]:
df['mattr'] = df['text'].apply(lambda x: LexicalRichness(x).mattr(window_size=25))

In [7]:
df['FK']=df['text'].apply(lambda x: textstat.flesch_kincaid_grade(x))

In [8]:
def tokenize(data):
    return nltk.word_tokenize(data)
def get_counts(tokens):
    counts=Counter()
    for token in tokens:
        counts[token]+=1
    return counts

In [9]:
def calculate_tc(testText): 
    tokens=tokenize(testText)
    token_counts=get_counts(tokens)
    freq_df=pd.DataFrame.from_dict(token_counts, orient='index', columns=['frequency']).reset_index().sort_values(by='frequency', ascending=False)
    freq_df['x']=1
    freq_df['rank']=freq_df.groupby('x').cumcount()+1
    fr1=freq_df.iloc[0,1]
    fr2=freq_df.iloc[1,1]
    r1=freq_df.iloc[0,3]
    r2=freq_df.iloc[1,3]
    h_point=((fr1*r2)-(fr2*r1))/(r2-r1+fr1-fr2)
    freq_df['TW']=np.where(freq_df['rank']<h_point, 2*(((h_point-freq_df['rank'])*(freq_df['frequency']))/((h_point*(h_point-1)*fr1))), 0)
    return freq_df['TW'].sum()

In [10]:
df['TC']=df['text'].apply(lambda x: calculate_tc(x))

In [11]:
trump=df[df['candidate']=='trump']

In [12]:
trump['rally']=np.where(trump['event_type']=='rally', 1, 0)

C:\Users\pcrrt\AppData\Local\Temp/ipykernel_33268/3625745240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trump['rally']=np.where(trump['event_type']=='rally', 1, 0)


In [13]:
trump_rallies=trump[trump['rally']==1]
trump_not_rallies=trump[trump['rally']==0]

In [14]:
ttest_ind(trump_rallies.TC, trump_not_rallies.TC, equal_var = False)

Ttest_indResult(statistic=-1.454314527744665, pvalue=0.14720579263877812)

In [27]:
mannwhitneyu(trump_rallies.TC, trump_not_rallies.TC)

MannwhitneyuResult(statistic=5898.5, pvalue=0.027675458769936635)

In [15]:
ttest_ind(trump_rallies.FK, trump_not_rallies.FK, equal_var = False)

Ttest_indResult(statistic=1.8718240800328838, pvalue=0.06257251549703144)

In [24]:
mannwhitneyu(trump_rallies.FK, trump_not_rallies.FK)

MannwhitneyuResult(statistic=4117.5, pvalue=4.711121496728959e-08)

In [16]:
ttest_ind(trump_rallies.mattr, trump_not_rallies.mattr, equal_var = False)

Ttest_indResult(statistic=-12.521639733001662, pvalue=3.0285404273506507e-26)

In [28]:
mannwhitneyu(trump_rallies.mattr, trump_not_rallies.mattr)

MannwhitneyuResult(statistic=1417.5, pvalue=3.515505442719496e-26)

In [17]:
trump.groupby('rally')[['mattr', 'TC', 'FK']].describe()

mattr                                                              \
       count      mean       std       min       25%       50%       75%   
rally                                                                      
0      114.0  0.852705  0.020288  0.782039  0.840668  0.850485  0.864404   
1      121.0  0.825397  0.011773  0.797862  0.816658  0.825332  0.830577   

                    TC            ...                         FK              \
            max  count      mean  ...       75%       max  count        mean   
rally                             ...                                          
0      0.904532  114.0  0.694871  ...  0.804192  0.912281  114.0  209.300000   
1      0.877199  121.0  0.662938  ...  0.781611  0.894395  121.0  393.685124   

                                                 
              std  min  25%  50%    75%     max  
rally                                            
0      624.076809  1.7  5.0  6.4  8.725  2488.7  
1      872.191811  2.9  4.0  4.7  5.500  2502.8  

[2 rows x 24 columns]

In [20]:
trump.event_type.value_counts()

rally        121
speech       104
debate         5
interview      5
Name: event_type, dtype: int64

In [21]:
trump.year.value_counts()

2020    119
2018     53
2019     40
2017     16
2021      4
2016      3
Name: year, dtype: int64

In [22]:
#only certain frequencies, stop words - LIWC - 70-80 features. Mispelled - not going to be able to find it
#topic modeling - dimensionality reduction
#most relevant words for each topic - give an identity to each topic - topic modeling to predict features for each
#5 topics - how prominent the topic is within the document = use that score as the feature - almost 30 x 5 - good row to column ratio
#For every feature - 6 rows - 
#sequence embedding for each speech - could cluster - look for certain outliers - one particular rally speech
#visualize in a two d pot
#t-ste - condense to two-d ste - interactive 2-d scatter plot - certain keywords
#pattern you can see from the sequence embeddings - color the dots - see that the red dots - 
#bert topic modeling 
#tfid - word threshold - top 1000 words - LIWC version - convert every word to LIWC features
#ngram=(2,2) - bigrams, max_df - do not only include words show up in 50% or fewer documents - 200 documents or less.
#min_df - only tokenize words that show up in at least 5 documents
#max_features - only 1000 most popular words! 
#binary - if word in the document or not
#tfid - see words that bring value - stop words - show up in many different documents
#low tfid - stopwords - are not labeled stopwords - find a way to help reduce the number of tokens - pass that into tfid
#once you fit tfidf - find the tokens that have a high tf-idf.
#hate speech/dehumanization - 
#https://towardsdatascience.com/kl-divergence-python-example-b87069e4b810
#https://towardsdatascience.com/interactive-topic-modeling-with-bertopic-1ea55e7d73d8
#https://cs.stanford.edu/people/wmorgan/sigtest.pdf